# fbiPy - Prisoners in the US
----

## Dataset: FBI

In [188]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
import json
# Import API key
from config import apikey
import ctypes  # An included library with Python install.
import datetime

In [199]:
def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)

def append_dict_to_df(current_dict, current_index, current_df):
    single_df = pd.DataFrame(current_dict, index=[current_index])
    if current_df.empty:
        current_df = single_df.copy()
    else:
        current_df = current_df.append(single_df)        
    current_index += 1
    return current_df,current_index

def get_offender_data(crime_type,variable,agency, year):
    endpoint = f"api/nibrs/{crime_type}/offender/agencies/{agency}/{variable}"
    done = False
    max_attempts = 5
    current_attemp = 1
    wait_time = 5
    df = pd.DataFrame()
    # Build query URL
    query_url = f"{base_url}{endpoint}?api_key={apikey}"
    # Call API and get response
    while (not done) & (current_attemp <= max_attempts):
        try:
            response = requests.get(query_url)
            if response.status_code != 404:
                data_json = response.json()
                df_idx = 0
                if len(data_json['data'])>0:
                    for data_nbr in range(len(data_json['data'])):
                        my_dict = data_json['data'][data_nbr]
                        df,df_idx = append_dict_to_df(my_dict, df_idx, df)
                    if len(df)>0:
                        df = df.loc[df['data_year'] == year]   
            done = True
            break
        except ConnectionAbortedError as err: 
            print(err)
            print(f"Attempt # {current_attemp} failed. Waiting {wait_time} seconds before next attempt.")
            time.sleep(wait_time)
            current_attemp =+ 1    
    return df

In [190]:
def build_scatter_plot(df, colx, coly, title, xlabel, ylabel, file):
    # Build a scatter plot
    plt.scatter(df[colx], df[coly], marker="o")
    # Incorporate the other graph properties
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.grid(True)
    # Save the figure
    plt.savefig(file)
    # Show plot
    plt.show()
    
def build_linear_regression(df, colx, coly, title, xlabel, ylabel, file, le_x, le_y, r_x, r_y):
    # Perform a linear regression on coly vs. colx
    x_values = df[colx]
    y_values = df[coly]
    # Calculate the line equation using linear regression function
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Plot original data using Scatter type
    plt.scatter(x_values,y_values, label='original data')
    # Plot fitted line using Line type
    plt.plot(x_values,regress_values,"r-", label='fitted line')
    # Add line equation to plot
    plt.annotate(line_eq,(le_x,le_y),fontsize=15,color="red")
    # Set title, xlabel, ylabel and legend
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    # Show r-squared value
    r2_latex = r'$r^2 = $'
    r2_value = f'{r2_latex}{round(rvalue**2,4)}'
    plt.annotate(r2_value,(r_x,r_y),fontsize=15,color="green")
    print(f"The r-value is: {rvalue}")
    # Save the figure
    plt.savefig(file)
    # Show plot
    plt.show()

In [191]:
# Config information.
base_url = "https://api.usa.gov/crime/fbi/sapi/"
format_type = "json"

In [78]:
# AGENCIES
endpoint = "api/agencies"

# Build query URL
query_url = f"{base_url}{endpoint}?api_key={apikey}"

# Call API and get response
response = requests.get(query_url)
data_json = response.json()
agencies_df = pd.DataFrame()
idx = 0
for state in data_json:
#     print(state)
    for agency in data_json[state]:
#         print(agency)
        agency_dict = data_json[state][agency]
        agencies_df,idx = append_dict_to_df(agency_dict, idx, agencies_df)

,ori,agency_name,agency_type_name,state_name,state_abbr,division_name,region_name,region_desc,county_name,nibrs,latitude,longitude,nibrs_start_date
0,HI0010000,Hawaii Police Department County Sheriff's Office,County,Hawaii,HI,Pacific,West,Region IV,HAWAII,False,19.7167,-155.087,None
1,HI0050000,Maui Police Department County Sheriff's Office,County,Hawaii,HI,Pacific,West,Region IV,MAUI,False,20.8873,-156.488,None
2,HI0020000,Honolulu Police Department,City,Hawaii,HI,Pacific,West,Region IV,HONOLULU,True,21.3043,-157.851,01/01/2018
3,HI0040000,Kauai Police Department County Sheriff's Office,County,Hawaii,HI,Pacific,West,Region IV,KAUAI,False,22.012,-159.706,None
4,DE0030500,Milford Police Department,City,Delaware,DE,South Atlantic,South,Region III,KENT; SUSSEX,True,38.9161,-75.4216,01/01/2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18663,TN0200000,Decatur County Sheriff's Office,County,Tennessee,TN,East South Central,South,Region III,DECATUR,True,35.6034,-88.1074,11/01/1997
18664,TN0070100,Jellico Police Department,City,Tennessee,TN,East South Central,South,Region III,CAMPBELL,True,36.587,-84.1301,06/01/1998
18665,DCMPD0000,Washington Police Department,City,District of Columbia,DC,South Atlantic,South,Region III,,False,38.8948,-77.0152,None
18666,DCMTP0000,Metro Transit Police,Other,District of Columbia,DC,South Atlantic,South,Region III,,True,38.8974,-77.0189,01/01/2000


In [157]:
# Output File (CSV)
output_data_file = "output_data/agencies.csv"

# Export file as a CSV
agencies_df.to_csv(output_data_file, index=False, header=True)   
agencies_df

,ori,agency_name,agency_type_name,state_name,state_abbr,division_name,region_name,region_desc,county_name,nibrs,latitude,longitude,nibrs_start_date
0,HI0010000,Hawaii Police Department County Sheriff's Office,County,Hawaii,HI,Pacific,West,Region IV,HAWAII,False,19.7167,-155.087,None
1,HI0050000,Maui Police Department County Sheriff's Office,County,Hawaii,HI,Pacific,West,Region IV,MAUI,False,20.8873,-156.488,None
2,HI0020000,Honolulu Police Department,City,Hawaii,HI,Pacific,West,Region IV,HONOLULU,True,21.3043,-157.851,01/01/2018
3,HI0040000,Kauai Police Department County Sheriff's Office,County,Hawaii,HI,Pacific,West,Region IV,KAUAI,False,22.012,-159.706,None
4,DE0030500,Milford Police Department,City,Delaware,DE,South Atlantic,South,Region III,KENT; SUSSEX,True,38.9161,-75.4216,01/01/2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18663,TN0200000,Decatur County Sheriff's Office,County,Tennessee,TN,East South Central,South,Region III,DECATUR,True,35.6034,-88.1074,11/01/1997
18664,TN0070100,Jellico Police Department,City,Tennessee,TN,East South Central,South,Region III,CAMPBELL,True,36.587,-84.1301,06/01/1998
18665,DCMPD0000,Washington Police Department,City,District of Columbia,DC,South Atlantic,South,Region III,,False,38.8948,-77.0152,None
18666,DCMTP0000,Metro Transit Police,Other,District of Columbia,DC,South Atlantic,South,Region III,,True,38.8974,-77.0189,01/01/2000


In [200]:
# TEST: single Agency
crime_type = "property-crime"
variable = "age"
year = 2019
current_agency = "HI0020000"
sub_df = get_offender_data(crime_type,variable,current_agency,year)
sub_df

,value,data_year,month_num,key
1,2,2019,0,0-9
3,602,2019,0,10-19
5,1063,2019,0,20-29
7,932,2019,0,30-39
9,520,2019,0,40-49
11,355,2019,0,50-59
13,131,2019,0,60-69
15,23,2019,0,70-79
17,3,2019,0,80-89
19,0,2019,0,90-Older


In [202]:
# Pull data for Variable: AGE and Year: 2019
start_time = datetime.datetime.now()
print(f'Process Start Time: {start_time}')
crime_type_list = ["property-crime", "violent-crime"]
variable = "age"
# year_list = [2015, 2016, 2017, 2018, 2019]
year_list = [2017]
agency_crimes_stat = pd.DataFrame()
idx = 0
total_agencies = len(agencies_df)
for index, row in agencies_df.iterrows():
    current_agency = row['ori']
    print(f"Agency {row['ori']} #{index+1}/{total_agencies}")
    for crime_type in crime_type_list:
        for year in year_list:
            sub_df = get_offender_data(crime_type,variable,current_agency,year)
            if len(sub_df)>0:
                print(f"          CrimeType: {crime_type}, Year: {year}, Processing Data")
                total_crimes = 0
                for i in range(len(sub_df)):
                    total_crimes += sub_df.iloc[i,0]
                print(f"                    Total crimes {total_crimes}, Year: {year}")
                my_dict = {
                    'year': year,
                    "crime_type": crime_type,
                    'agency': current_agency,
                    'agency_name': row['agency_name'],
                    'county': row['county_name'],
                    'state': row['state_abbr'],
                    'state_name': row['state_name'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'age 0-9': sub_df.iloc[0,0],
                    'age 10-19': sub_df.iloc[1,0],
                    'age 20-29': sub_df.iloc[2,0],
                    'age 30-39': sub_df.iloc[3,0],
                    'age 40-49': sub_df.iloc[4,0],
                    'age 50-59': sub_df.iloc[5,0],
                    'age 60-69': sub_df.iloc[6,0],
                    'age 70-79': sub_df.iloc[7,0],
                    'age 80-89': sub_df.iloc[8,0],
                    'age 90-Older': sub_df.iloc[9,0],
                    'age Unknown': sub_df.iloc[10,0],
                    'total_crimes': total_crimes
                }
                agency_crimes_stat,idx = append_dict_to_df(my_dict, idx, agency_crimes_stat)
            else:
                print(f"          CrimeType: {crime_type}, Year: {year}, No Data")
# Output File (CSV)
output_data_file = "output_data/agency_crimes_stat.csv"
# Export file as a CSV
agency_crimes_stat.to_csv(output_data_file, index=False, header=True)
end_time = datetime.datetime.now()
print(f'Process End Time: {end_time}')
elapsed_time = end_time - start_time
print(f'Elapsed Time: {elapsed_time}')
# Show DataFrame
agency_crimes_stat

Process Start Time: 2021-02-09 17:32:44.110589
Agency HI0010000 #1/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency HI0050000 #2/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency HI0020000 #3/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency HI0040000 #4/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency DE0030500 #5/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 550, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 105, Year: 2017
Agency DE0030300 #6/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 171, Year: 2017
    

          CrimeType: violent-crime, Year: 2017, No Data
Agency DE003500X #38/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 1, Year: 2017
          CrimeType: violent-crime, Year: 2017, No Data
Agency DE0013300 #39/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency DE0011800 #40/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 9, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 3, Year: 2017
Agency DE0030800 #41/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 461, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 75, Year: 2017
Agency DE0030600 #42/18668
          CrimeType: property-crime, Year: 2017, Processing Data
              

          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 38, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 3, Year: 2017
Agency TX0840200 #77/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX1017000 #78/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0202800 #79/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0290100 #80/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0680200 #81/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0450100 #82/18668
          CrimeType: property-crime, Year: 2017, 

          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 129, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 46, Year: 2017
Agency TX1013900 #131/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0930000 #132/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0141100 #133/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0701000 #134/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0370100 #135/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 203, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
    

          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 6, Year: 2017
Agency TX2000000 #183/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0610500 #184/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX1081400 #185/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0700100 #186/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0430700 #187/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX0310200 #188/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TX1261200 #189/18668
          Crime

...

          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 2, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 1, Year: 2017
Agency TN0190300 #18543/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 33, Year: 2017
          CrimeType: violent-crime, Year: 2017, No Data
Agency TN0160000 #18544/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 413, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 74, Year: 2017
Agency TN0930000 #18545/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 359, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 43, Year: 2017
Agency TN0680300 #18546/18668
          CrimeType: property-crime,

          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 361, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 54, Year: 2017
Agency TN0400000 #18579/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 435, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 45, Year: 2017
Agency TN0240800 #18580/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 30, Year: 2017
          CrimeType: violent-crime, Year: 2017, No Data
Agency TN0070400 #18581/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 93, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 5, Year: 2017
Agency TN0540300 #18582/18668
          CrimeType: property-crime

          CrimeType: violent-crime, Year: 2017, No Data
Agency TN0280200 #18613/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, No Data
Agency TN0620200 #18614/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 290, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 24, Year: 2017
Agency TN0060300 #18615/18668
          CrimeType: property-crime, Year: 2017, No Data
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 1, Year: 2017
Agency TN0840100 #18616/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 521, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 115, Year: 2017
Agency TN0331600 #18617/18668
          CrimeType: property-crime, Year: 2017, Processing 

          CrimeType: violent-crime, Year: 2017, No Data
Agency TN079279E #18649/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 21, Year: 2017
          CrimeType: violent-crime, Year: 2017, No Data
Agency TN0780100 #18650/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 269, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 37, Year: 2017
Agency TN0350100 #18651/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 162, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total crimes 68, Year: 2017
Agency TN0830200 #18652/18668
          CrimeType: property-crime, Year: 2017, Processing Data
                    Total crimes 273, Year: 2017
          CrimeType: violent-crime, Year: 2017, Processing Data
                    Total

,year,crime_type,agency,agency_name,county,state,state_name,latitude,longitude,age 0-9,...,age 20-29,age 30-39,age 40-49,age 50-59,age 60-69,age 70-79,age 80-89,age 90-Older,age Unknown,total_crimes
0,2017,property-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,151,89,67,47,6,2,0,0,113,550
1,2017,violent-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,30,14,4,3,0,0,0,0,20,105
2,2017,property-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,1,...,58,24,9,5,1,3,1,0,42,171
3,2017,violent-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,0,...,22,9,4,1,0,0,0,0,6,66
4,2017,property-crime,DE0011900,Dover Fire Marshal,KENT,DE,Delaware,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12080,2017,violent-crime,TN0200000,Decatur County Sheriff's Office,DECATUR,TN,Tennessee,35.603422,-88.107384,0,...,4,5,2,0,1,0,0,0,1,15
12081,2017,property-crime,TN0070100,Jellico Police Department,CAMPBELL,TN,Tennessee,36.586983,-84.130090,0,...,6,30,11,1,0,0,0,0,17,70
12082,2017,violent-crime,TN0070100,Jellico Police Department,CAMPBELL,TN,Tennessee,36.586983,-84.130090,0,...,0,3,1,0,1,0,0,0,0,5
12083,2017,property-crime,DCMTP0000,Metro Transit Police,,DC,District of Columbia,38.897390,-77.018940,2,...,104,40,17,20,2,0,0,0,201,720


In [203]:
# # Output File (CSV)
# output_data_file = "output_data/agency_crimes_stat.csv"
# # Export file as a CSV
# agency_crimes_stat.to_csv(output_data_file, index=False, header=True)
# Show DataFrame
agency_crimes_stat

,year,crime_type,agency,agency_name,county,state,state_name,latitude,longitude,age 0-9,...,age 20-29,age 30-39,age 40-49,age 50-59,age 60-69,age 70-79,age 80-89,age 90-Older,age Unknown,total_crimes
0,2017,property-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,151,89,67,47,6,2,0,0,113,550
1,2017,violent-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,30,14,4,3,0,0,0,0,20,105
2,2017,property-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,1,...,58,24,9,5,1,3,1,0,42,171
3,2017,violent-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,0,...,22,9,4,1,0,0,0,0,6,66
4,2017,property-crime,DE0011900,Dover Fire Marshal,KENT,DE,Delaware,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12080,2017,violent-crime,TN0200000,Decatur County Sheriff's Office,DECATUR,TN,Tennessee,35.603422,-88.107384,0,...,4,5,2,0,1,0,0,0,1,15
12081,2017,property-crime,TN0070100,Jellico Police Department,CAMPBELL,TN,Tennessee,36.586983,-84.130090,0,...,6,30,11,1,0,0,0,0,17,70
12082,2017,violent-crime,TN0070100,Jellico Police Department,CAMPBELL,TN,Tennessee,36.586983,-84.130090,0,...,0,3,1,0,1,0,0,0,0,5
12083,2017,property-crime,DCMTP0000,Metro Transit Police,,DC,District of Columbia,38.897390,-77.018940,2,...,104,40,17,20,2,0,0,0,201,720


## Plotting the Data


### x_values vs. y_values Plot

In [ ]:
# # build_scatter_plot(df, colx, coly, title, xlabel, ylabel, file)
# build_scatter_plot(agency_crimes_stat, county, total_crimes, "County vs Total crimes", "County", "Total crimes", 
#                    "output_data/county_total_crimes.png")

In [ ]:
# build_linear_plot(df, colx, coly, title, xlabel, ylabel, file)
# build_linear_plot(agency_crimes_stat, county, total_crimes, "County vs Total crimes", "County", "Total crimes", 
#                    "output_data/county_total_crimes.png")

## Linear Regression

In [ ]:
# build_linear_regression(df, colx, coly, title, xlabel, ylabel, file, le_x, le_y, r_x, r_y)